In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\Ali\AppData\Local\Temp\ipykernel_5360\265554930.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# data leading and preparing

In [2]:
data = pd.read_csv('./dft-road-casualty-statistics-casualty-provisional-mid-year-unvalidated-2022.csv', index_col="accident_index")
data.head()

,status,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
accident_index,,,,,,,,,,,,,,,,,,,
2022070151244,Unvalidated,2022,070151244,2,1,1,2,46,8,3,0,0,0,0,0,9,1,9,E01033378
2022070152668,Unvalidated,2022,070152668,1,1,1,1,30,6,3,0,0,0,0,0,9,1,2,E01018551
2022070154696,Unvalidated,2022,070154696,1,1,1,2,58,9,3,0,0,0,0,0,9,1,10,E01006134
2022070154696,Unvalidated,2022,070154696,2,3,1,2,78,11,3,0,0,0,0,0,9,2,10,E01018724
2022070154696,Unvalidated,2022,070154696,3,2,1,1,63,9,3,0,0,0,0,0,9,3,7,W01000578


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61352 entries, 2022070151244 to 2022461159841
Data columns (total 17 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   accident_reference                  61352 non-null  object
 1   vehicle_reference                   61352 non-null  int64 
 2   casualty_reference                  61352 non-null  int64 
 3   casualty_class                      61352 non-null  int64 
 4   sex_of_casualty                     61352 non-null  int64 
 5   age_of_casualty                     61352 non-null  int64 
 6   age_band_of_casualty                61352 non-null  int64 
 7   casualty_severity                   61352 non-null  int64 
 8   pedestrian_location                 61352 non-null  int64 
 9   pedestrian_movement                 61352 non-null  int64 
 10  car_passenger                       61352 non-null  int64 
 11  bus_or_coach_passenger              613

# EDA

## status column

In [5]:
data['status'].unique()

array(['Unvalidated'], dtype=object)

Every value in "status" column, is "Unvalidated".
So this column has nothing to tell and we can drop it.

In [7]:
data = data.drop(['status'], axis=1)

## accident_year column

In [8]:
data.accident_year.unique()

array([2022], dtype=int64)

Every value in "accident_year" column, is "2022".
So this column has nothing to tell and we can drop it.

In [9]:
data = data.drop(['accident_year'], axis=1)

## accident_reference

In [16]:
data.accident_reference.unique()
# no problem with unique values
data.accident_reference.nunique()
# there is 48283 kind of values in this column

# trying to turn it to numeric tyupe
for value in data.accident_reference:
    try:
        float(value)
    except:
        print(value)
        break

170H10152


this column is a kind of code.
the nunique id too large to decode it.
it's almost different for every column.
so we refuse using it, because there is no information in this column.

In [19]:
data = data.drop(['accident_reference'], axis=1)

In [23]:
duplicated_indexes = data.index[data.index.duplicated()]

In [24]:
duplicated_indexes

Index(['2022070154696', '2022070154696', '2022070154696', '2022070154696',
       '2022070154696', '2022070178065', '2022070208889', '2022070230394',
       '2022070230394', '2022070231976',
       ...
       '2022451152630', '2022451153071', '2022451161902', '2022461149113',
       '2022461149684', '2022461153688', '2022461153714', '2022461154159',
       '2022461155125', '2022461159841'],
      dtype='object', name='accident_index', length=13069)

In [25]:
data.loc[2022461154159]

KeyError: 2022461154159